<a href="https://colab.research.google.com/github/Sivachandran5710/HCL-training/blob/main/Paper-1(Industrial_vision)/unit-2/Resnet_%26_Mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------------------------------
# 1. Setup (Colab has TF pre-installed)
# -------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow version:", tf.__version__)

# -------------------------------------
# 2. Load a sample dataset (replace with your own)
# Colab provides tf.keras.datasets, here we use CIFAR-10 for demo
# -------------------------------------
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.cifar10.load_data()

# Normalize images
x_train = x_train.astype("float32") / 255.0
x_val = x_val.astype("float32") / 255.0

# Resize images to 224x224 (needed for ResNet/MobileNet)
resize_layer = tf.keras.Sequential([
    layers.Resizing(224, 224)
])

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).map(lambda x, y: (resize_layer(x), y))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32).map(lambda x, y: (resize_layer(x), y))

# -------------------------------------
# 3. Transfer Learning with ResNet50
# -------------------------------------
base_resnet = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_resnet.trainable = False  # freeze backbone

resnet_model = models.Sequential([
    base_resnet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")  # CIFAR-10 has 10 classes
])

resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Training ResNet50...")
resnet_model.fit(train_ds, validation_data=val_ds, epochs=3)


# -------------------------------------
# 4. Fine-tuning (optional: unfreeze last few layers)
# -------------------------------------
base_resnet.trainable = True
for layer in base_resnet.layers[:-30]:
    layer.trainable = False

resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # lower LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Fine-tuning ResNet50...")
resnet_model.fit(train_ds, validation_data=val_ds, epochs=2)
# -------------------------------------
# 5. Save the trained model
# -------------------------------------

# Recommended: Save in the modern Keras v3 format
resnet_model.save('resnet_cifar10.keras')
print("Model saved in .keras format.")

TensorFlow version: 2.19.0
Training ResNet50...
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 185s 112ms/step - accuracy: 0.1076 - loss: 2.3702 - val_accuracy: 0.1276 - val_loss: 2.2927
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 154s 98ms/step - accuracy: 0.1330 - loss: 2.2907 - val_accuracy: 0.1646 - val_loss: 2.2787
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 155s 99ms/step - accuracy: 0.1513 - loss: 2.2746 - val_accuracy: 0.2115 - val_loss: 2.2561
Fine-tuning ResNet50...
Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 239s 141ms/step - accuracy: 0.3190 - loss: 1.8917 - val_accuracy: 0.4134 - val_loss: 1.6160
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 211s 135ms/step - accuracy: 0.4816 - loss: 1.4549 - val_accuracy: 0.4704 - val_loss: 1.4682
Model saved in .keras format.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/My Drive/images/cat.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted class: 3


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/My Drive/images/aero.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted class: 0


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/My Drive/images/auto.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted class: 1


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/My Drive/images/deer.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted class: 6


In [ ]:
# -------------------------------------
# 1. Setup
# -------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow version:", tf.__version__)

# -------------------------------------
# 2. Load dataset (CIFAR-10)
# -------------------------------------
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.cifar10.load_data()

# Normalize images
x_train = x_train.astype("float32") / 255.0
x_val = x_val.astype("float32") / 255.0

# Resize to 224x224 for MobileNetV2
resize_layer = tf.keras.Sequential([
    layers.Resizing(224, 224)
])

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).map(lambda x, y: (resize_layer(x), y))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32).map(lambda x, y: (resize_layer(x), y))

# -------------------------------------
# 3. Transfer Learning with MobileNetV2
# -------------------------------------
# Load pre-trained MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base model
base_model.trainable = False

# Build the full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Training MobileNetV2...")
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

# -------------------------------------
# 4. Fine-tuning: unfreeze top layers of base model
# -------------------------------------
# Now make the base model trainable
base_model.trainable = True

# Freeze all layers except the last 30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Fine-tuning MobileNetV2...")
history_fine = model.fit(train_ds, validation_data=val_ds, epochs=2)

# -------------------------------------
# 5. Save the trained model
# -------------------------------------
model.save('mobilenet_cifar10.keras')
print("Model saved in .keras format.")

TensorFlow version: 2.19.0
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training MobileNetV2...
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 90s 47ms/step - accuracy: 0.5508 - loss: 1.3282 - val_accuracy: 0.7747 - val_loss: 0.6549
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 54s 35ms/step - accuracy: 0.7503 - loss: 0.7210 - val_accuracy: 0.7942 - val_loss: 0.5888
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 56s 36ms/step - accuracy: 0.7803 - loss: 0.6395 - val_accuracy: 0.8023 - val_loss: 0.5628
Fine-tuning MobileNetV2...
Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 99s 52ms/step - accuracy: 0.7107 - loss: 0.8415 - val_accuracy: 0.8232 - val_loss: 0.5113
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 67s 43ms/step - accuracy: 0.8197 - loss: 0.5343 - val_accuracy: 0.8408 - val_loss: 0.4555
Model saved in .keras format.
